In [1]:
!pip install xgboost
!pip install nbformat

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
import librosa
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis

INDEX = 13
DATA_DIR = './new-test-data/test-mfcc-v2'
DATA2_DIR = './new-test-data/test-mfcc-v2-copy'

def aggregate_mfcc_selective(mfcc_data):
    mfcc_selected = mfcc_data[:20, :] # Select first 20 MFCCs
    
    mfcc_mean = np.mean(mfcc_selected, axis=1)
    mfcc_std = np.std(mfcc_selected, axis=1)
    mfcc_max = np.max(mfcc_selected, axis=1)
    mfcc_min = np.min(mfcc_selected, axis=1)
    
    features = np.concatenate([mfcc_mean, mfcc_std, mfcc_max, mfcc_min])
    return features

def aggregate_mfcc_selective2(mfcc_data):
    mfcc_selected = mfcc_data[:3, :] # Select first 20 MFCCs
    
    mfcc_mean = np.mean(mfcc_selected, axis=1)
    mfcc_std = np.std(mfcc_selected, axis=1)
    mfcc_max = np.max(mfcc_selected, axis=1)
    mfcc_min = np.min(mfcc_selected, axis=1)
    
    features = np.concatenate([mfcc_mean, mfcc_std, mfcc_max, mfcc_min])
    return features

# Initialize lists to store features, file names, and labels
mfcc_all_songs = []
file_names = []
generated_features = []
labels = []

# total number of files in the directory
total_files = len(os.listdir(DATA_DIR))
index = 1

# Iterate over all files in the specified directory
for file_name in os.listdir(DATA_DIR):
    file_path = os.path.join(DATA_DIR, file_name)
    file_path2 = os.path.join(DATA2_DIR, file_name)
    print(file_path, file_path2)
    mfcc_data = pd.read_csv(file_path, header=None).values
    mfcc_data = mfcc_data[:21]
    mfcc_data2 = pd.read_csv(file_path2, header=None).values
    # Compute aggregated MFCC features
    aggregated_features = aggregate_mfcc_selective(mfcc_data)
    aggregated_features2 = aggregate_mfcc_selective2(mfcc_data2)

    skewness = skew(mfcc_data, axis=1)
    kurt = kurtosis(mfcc_data, axis=1)
    range_max_min = np.ptp(mfcc_data, axis=1)

    delta_mfcc = librosa.feature.delta(mfcc_data, order=1)
    delta_delta_mfcc = librosa.feature.delta(mfcc_data, order=2)

    delta_mean = np.mean(delta_mfcc, axis=1)
    delta_std = np.std(delta_mfcc, axis=1)
    delta_max = np.max(delta_mfcc, axis=1)
    delta_min = np.min(delta_mfcc, axis=1)

    delta_delta_mean = np.mean(delta_mfcc, axis=1)
    delta_delta_std = np.std(delta_mfcc, axis=1)
    delta_delta_max = np.max(delta_mfcc, axis=1)
    delta_delta_min = np.min(delta_mfcc, axis=1)

    delta_mfcc2 = librosa.feature.delta(mfcc_data2, order=1)
    delta_delta_mfcc2 = librosa.feature.delta(mfcc_data2, order=2)

    delta_mean2 = np.mean(delta_mfcc2, axis=1)
    delta_std2 = np.std(delta_mfcc2, axis=1)
    delta_max2 = np.max(delta_mfcc2, axis=1)
    delta_min2 = np.min(delta_mfcc2, axis=1)

    delta_delta_mean2 = np.mean(delta_mfcc2, axis=1)
    delta_delta_std2 = np.std(delta_mfcc2, axis=1)
    delta_delta_max2 = np.max(delta_mfcc2, axis=1)
    delta_delta_min2 = np.min(delta_mfcc2, axis=1)

    PCA_mfcc = PCA(n_components=5)
    PCA_mfcc.fit(mfcc_data.T)
    PCA_mfcc_features = PCA_mfcc.components_.flatten()

    # Compile all features into a single vector
    features = np.concatenate([
        aggregated_features.flatten(),
        range_max_min.flatten(),
        delta_mean.flatten(),
        delta_std.flatten(),
        delta_max.flatten(),
        delta_min.flatten(),
        delta_mean2.flatten(),
        delta_std2.flatten(),
        delta_max2.flatten(),
        delta_min2.flatten(),
        PCA_mfcc_features.flatten(),
    ])

    # Determine label based on filename pattern
    if file_name.startswith('Asha-Bhosle'):
        label = 'Asha Bhosale'
    elif file_name.startswith('Kishore-Kumar'):
        label = 'Kishore Kumar'
    elif file_name.startswith('Lavni'):
        label = 'Lavni'
    elif file_name.startswith('Michael-Jackson'):
        label = 'Michael Jackson'
    elif file_name.startswith('Jana-Gana-Mana'):
        label = 'Jana-Gana-Mana'
    elif file_name.startswith('Bhavgeet'):
        label = 'Bhavgeet'
        
    print(f'Processed {file_name}, {index}/{total_files}')
    # print(features.shape)
    generated_features.append(features)
    file_names.append(file_name)
    labels.append(label)
    index += 1

# Create DataFrame and save to CSV
generated_features = np.vstack(generated_features)
total_features = generated_features.shape[1]
feature_columns = [f'feature_{i}' for i in range(total_features)]

total_data_df = pd.DataFrame(generated_features, columns=feature_columns)
total_data_df.insert(0, 'File', file_names)
total_data_df.insert(1, 'Label', labels)

# Save generated features to 'features_generated.csv'
total_data_df.to_csv(f'features_generated_{INDEX}.csv', index=False)


./new-test-data/test-mfcc-v2/Asha-Bhosle-23.csv ./new-test-data/test-mfcc-v2-copy/Asha-Bhosle-23.csv
Processed Asha-Bhosle-23.csv, 1/285
./new-test-data/test-mfcc-v2/Michael-Jackson-55.csv ./new-test-data/test-mfcc-v2-copy/Michael-Jackson-55.csv
Processed Michael-Jackson-55.csv, 2/285
./new-test-data/test-mfcc-v2/Kishore-Kumar-63.csv ./new-test-data/test-mfcc-v2-copy/Kishore-Kumar-63.csv
Processed Kishore-Kumar-63.csv, 3/285
./new-test-data/test-mfcc-v2/Asha-Bhosle-32.csv ./new-test-data/test-mfcc-v2-copy/Asha-Bhosle-32.csv
Processed Asha-Bhosle-32.csv, 4/285
./new-test-data/test-mfcc-v2/Lavni-20.csv ./new-test-data/test-mfcc-v2-copy/Lavni-20.csv
Processed Lavni-20.csv, 5/285
./new-test-data/test-mfcc-v2/Michael-Jackson-42.csv ./new-test-data/test-mfcc-v2-copy/Michael-Jackson-42.csv
Processed Michael-Jackson-42.csv, 6/285
./new-test-data/test-mfcc-v2/Asha-Bhosle-71.csv ./new-test-data/test-mfcc-v2-copy/Asha-Bhosle-71.csv
Processed Asha-Bhosle-71.csv, 7/285
./new-test-data/test-mfcc-v2/

In [4]:

#perform EDA here
#Correlation HeatMap
#Scatterplots and Boxplots


In [17]:

# Splitting the data into features (X) and labels (y)
X = total_data_df.drop(columns=['File', 'Label'])
y = total_data_df['Label']


In [6]:

# Splitting into training (70%) and test (30%) datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Dictionary to store models and results
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    #'SVM': SVC(kernel='rbf', C=1.0, probability=True), # SVC with probability=True for probability estimates
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1),
    'AdaBoost': AdaBoostClassifier(n_estimators=50),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(max_depth=10),
    'XGBoost': xgb.XGBClassifier(n_estimators=100, learning_rate=0.1),
    'SVC_linear': SVC(kernel='linear', probability=True),
    'SVC_rbf': SVC(kernel='rbf', probability=True),
    'RandomForest_1': RandomForestClassifier(min_samples_leaf=1),
    'RandomForest_3': RandomForestClassifier(min_samples_leaf=3),
    'RandomForest_5': RandomForestClassifier(min_samples_leaf=5),
    'NeuralNetwork_1': MLPClassifier(hidden_layer_sizes=(5)),
    'NeuralNetwork_2': MLPClassifier(hidden_layer_sizes=(5, 5)),
    'NeuralNetwork_3': MLPClassifier(hidden_layer_sizes=(5, 5, 5)),
    'NeuralNetwork_4': MLPClassifier(hidden_layer_sizes=(10))

}


In [9]:
import plotly.graph_objects as go

results = {}

# Training, prediction, and metric calculations
for model_name, model in models.items():
    print(f"\nTraining {model_name}...")
    model.fit(X_train, y_train_encoded)  # Train the model
    y_pred = model.predict(X_test)  # Predict on the test set
    
    # Storing metrics
    results[model_name] = {
        "Accuracy": accuracy_score(y_test_encoded, y_pred),
        "Precision": precision_score(y_test_encoded, y_pred, average='weighted'),
        "Recall": recall_score(y_test_encoded, y_pred, average='weighted'),
        "F1 Score": f1_score(y_test_encoded, y_pred, average='weighted'),
        "Confusion Matrix": confusion_matrix(y_test_encoded, y_pred),
        "Classification Report": classification_report(y_test_encoded, y_pred)
    }

    # Plotting the confusion matrix using Plotly with numbers displayed and square grid
    confusion_mat = results[model_name]["Confusion Matrix"]
    fig = go.Figure(data=go.Heatmap(
        z=confusion_mat,
        x=[f"Predicted {i}" for i in range(len(confusion_mat))],
        y=[f"Actual {i}" for i in range(len(confusion_mat))],
        colorscale="Blues",
        text=confusion_mat,  # Display numbers in each cell
        texttemplate="%{text}",  # Format to show text values
        showscale=True
    ))
    fig.update_layout(
        title=f"Confusion Matrix for {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="Actual Label",
        xaxis=dict(scaleanchor="y", scaleratio=1)  # Ensures square cells
    )
    fig.show()



Training Logistic Regression...


/home/nirav24/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Training K-Nearest Neighbors...



Training Random Forest...



Training Gradient Boosting...


...

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to 


Training NeuralNetwork_3...


/home/nirav24/.local/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/ni


Training NeuralNetwork_4...


/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/nirav24/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to 

In [10]:

# Display the results
for model_name, metrics in results.items():
    print(f"\nMetrics for {model_name}:\n")
    for metric, value in metrics.items():
        print(f"{metric}:\n{value}\n")



Metrics for Logistic Regression:

Accuracy:
0.6976744186046512

Precision:
0.7084214235377025

Recall:
0.6976744186046512

F1 Score:
0.6926289205070976

Confusion Matrix:
[[15  2 10  3]
 [ 1 19  0  2]
 [ 3  1 11  2]
 [ 2  0  0 15]]

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.50      0.59        30
           1       0.86      0.86      0.86        22
           2       0.52      0.65      0.58        17
           3       0.68      0.88      0.77        17

    accuracy                           0.70        86
   macro avg       0.70      0.72      0.70        86
weighted avg       0.71      0.70      0.69        86



Metrics for K-Nearest Neighbors:

Accuracy:
0.3372093023255814

Precision:
0.3361757732666249

Recall:
0.3372093023255814

F1 Score:
0.3355151972258832

Confusion Matrix:
[[15  6  5  4]
 [ 9  6  2  5]
 [ 4  3  3  7]
 [ 3  4  5  5]]

Classification Report:
              precision    recall  f1-score   supp

In [11]:
print("\nAll Models Accuracy:")
for model_name, metrics in results.items():
    print(f"{model_name}: {metrics['Accuracy']:.4f}")


All Models Accuracy:
Logistic Regression: 0.6977
K-Nearest Neighbors: 0.3372
Random Forest: 0.8721
Gradient Boosting: 0.8372
AdaBoost: 0.6860
Naive Bayes: 0.8256
Decision Tree: 0.8023
XGBoost: 0.8372
SVC_linear: 0.7674
SVC_rbf: 0.2093
RandomForest_1: 0.8488
RandomForest_3: 0.8372
RandomForest_5: 0.8605
NeuralNetwork_1: 0.1977
NeuralNetwork_2: 0.1977
NeuralNetwork_3: 0.2558
NeuralNetwork_4: 0.3488


In [12]:
print("\nAll Models Precison:")
for model_name, metrics in results.items():
    print(f"{model_name}: {metrics['Precision']:.4f}")


All Models Precison:
Logistic Regression: 0.7084
K-Nearest Neighbors: 0.3362
Random Forest: 0.8744
Gradient Boosting: 0.8413
AdaBoost: 0.7338
Naive Bayes: 0.8338
Decision Tree: 0.8160
XGBoost: 0.8433
SVC_linear: 0.7750
SVC_rbf: 0.0949
RandomForest_1: 0.8501
RandomForest_3: 0.8568
RandomForest_5: 0.8732
NeuralNetwork_1: 0.0391
NeuralNetwork_2: 0.0391
NeuralNetwork_3: 0.0654
NeuralNetwork_4: 0.1217


In [13]:
print("\nAll Models Recall:")
for model_name, metrics in results.items():
    print(f"{model_name}: {metrics['Recall']:.4f}")


All Models Recall:
Logistic Regression: 0.6977
K-Nearest Neighbors: 0.3372
Random Forest: 0.8721
Gradient Boosting: 0.8372
AdaBoost: 0.6860
Naive Bayes: 0.8256
Decision Tree: 0.8023
XGBoost: 0.8372
SVC_linear: 0.7674
SVC_rbf: 0.2093
RandomForest_1: 0.8488
RandomForest_3: 0.8372
RandomForest_5: 0.8605
NeuralNetwork_1: 0.1977
NeuralNetwork_2: 0.1977
NeuralNetwork_3: 0.2558
NeuralNetwork_4: 0.3488


In [14]:
print("\nAll Models f1-Score:")
for model_name, metrics in results.items():
    print(f"{model_name}: {metrics['F1 Score']:.4f}")


All Models f1-Score:
Logistic Regression: 0.6926
K-Nearest Neighbors: 0.3355
Random Forest: 0.8714
Gradient Boosting: 0.8374
AdaBoost: 0.6967
Naive Bayes: 0.8233
Decision Tree: 0.8050
XGBoost: 0.8373
SVC_linear: 0.7605
SVC_rbf: 0.1306
RandomForest_1: 0.8471
RandomForest_3: 0.8391
RandomForest_5: 0.8614
NeuralNetwork_1: 0.0653
NeuralNetwork_2: 0.0653
NeuralNetwork_3: 0.1042
NeuralNetwork_4: 0.1804


In [15]:
#show decision boundary
#ROC Curve

In [16]:
#identify the three songs of each category
#showcase the waveform for each of the three songs found for each category